In [106]:
# pip install PyPDF2

In [107]:
# pip install wheel

In [108]:
# pip install pandas

In [109]:
# pip install tiktoken

In [110]:
# pip install openai

In [111]:
# pip install tenacity

In [112]:
import re
# Helper function to clean the description!
def clean_description(description: str):
    if not description:
        return ""
    # remove unicode characters
    description = description.encode('ascii', 'ignore').decode()

    # remove punctuation
    # description = re.sub('[%s]' % re.escape(string.punctuation), ' ', description)

    # remove newlines
    description = description.replace('\n', ' ')
    description = description.replace('\\n', ' ')

    # clean up the spacing
    description = re.sub('\s{2,}', ' ', description)

    # remove urls
    #description = re.sub("https*\S+", " ", description)

    # remove newlines
    # description = description.replace("\n", " ")

    # remove all numbers
    #description = re.sub('\w*\d+\w*', '', description)

    # split on capitalized words
    # description = " ".join(re.split('(?=[A-Z])', description))

    # clean up the spacing again
    # description = re.sub('\s{2,}', " ", description)

    # make all words lowercase
    # description = description.lower()

    return description.strip()

In [113]:
from PyPDF2 import PdfReader
import os
import pandas as pd

# Get all pdf files in the pdf directory
texts = []
for file in os.listdir('./pdf/'):
    extractedText = ''
    reader = PdfReader('./pdf/' + file)
    for page in reader.pages:
        # extracting text from page
        extractedText += page.extract_text()
    texts.append(clean_description(extractedText))

for text in texts:
    print(text)

2023WARRANTY & MAINTENANCE GUIDE Printed in U.S.A. 9/22 21-TCS-16245Warranty & Maintenance Guide 2023 toyota.com 134116_21-TCS-16245 Toyota Warranty and Maintenance Guide - MY23 Venza COVER_R1.indd 1 134116_21-TCS-16245 Toyota Warranty and Maintenance Guide - MY23 Venza COVER_R1.indd 1 9/14/22 4:44 AM 9/14/22 4:44 AMSAFETY PRECAUTIONS Your Venza has both high-voltage DC and AC systems as well as a 12-volt system. Both DC and AC high voltage are very dangerous and can cause personal injury, severe burns, electric shock and even fatal injury. To avoid personal injury, please follow all caution labels attached to high-voltage parts. Do not touch or attempt to remove or replace any high-voltage (orange-colored) wiring and connectors. Do not touch the service plug, which is located under the rear seat. If an accident occurs, do not touch any high-voltage wiring, connectors or parts such as the inverter unit or hybrid vehicle battery assembly. Do not touch the potassium hydroxide electrolyte

In [114]:
import tiktoken

EMBEDDING_MODEL = 'text-embedding-ada-002'
MAX_INPUT_TOKENS = 8191
DELIMITER = '. '


def get_num_tokens(text: str, model: str = EMBEDDING_MODEL) -> int:
    # Return the number of tokens in a string
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


def split_string_in_half(string) -> 'list[str, str]':
    length = len(string)
    half = length // 2

    first_half = string[:half]
    second_half = string[half:]

    return [first_half, second_half]


def halved_by_delimiter(string: str, num_tokens: int, delimiter: str) -> 'list[str, str]':
    """Split a string in two, on a delimiter, trying to balance tokens on each side."""
    chunks = string.split(delimiter)
    if len(chunks) == 1:
        # no delimiter found so split in half
        return split_string_in_half(string)
    
    if len(chunks) == 2:
        return chunks # no need to search for halfway point
    else:
        halfway = num_tokens // 2
        best_diff = halfway
        for i, chunk in enumerate(chunks):
            left = delimiter.join(chunks[: i + 1])
            left_tokens = get_num_tokens(left)
            diff = abs(halfway - left_tokens)
            if diff >= best_diff:
                break
            else:
                best_diff = diff
        left = delimiter.join(chunks[:i])
        right = delimiter.join(chunks[i:])
        return [left, right]

# TODO rewrite not to use recursions
def chunk_text(text: str) -> 'list[str]':
    # split text into chunks
    num_tokens_in_string = get_num_tokens(text)

    if num_tokens_in_string <= MAX_INPUT_TOKENS:
        return [text]
    else:
        left, right = halved_by_delimiter(text, num_tokens_in_string, DELIMITER)
        # recurse on each half
        results = []
        for half in [left, right]:
            half_strings = chunk_text(half)
            results.extend(half_strings)
        return results


In [115]:
pdf_strings = []
for text in texts:
    pdf_strings.extend(chunk_text(text))

for string in pdf_strings:
    print(get_num_tokens(string), string)

6149 2023WARRANTY & MAINTENANCE GUIDE Printed in U.S.A. 9/22 21-TCS-16245Warranty & Maintenance Guide 2023 toyota.com 134116_21-TCS-16245 Toyota Warranty and Maintenance Guide - MY23 Venza COVER_R1.indd 1 134116_21-TCS-16245 Toyota Warranty and Maintenance Guide - MY23 Venza COVER_R1.indd 1 9/14/22 4:44 AM 9/14/22 4:44 AMSAFETY PRECAUTIONS Your Venza has both high-voltage DC and AC systems as well as a 12-volt system. Both DC and AC high voltage are very dangerous and can cause personal injury, severe burns, electric shock and even fatal injury. To avoid personal injury, please follow all caution labels attached to high-voltage parts. Do not touch or attempt to remove or replace any high-voltage (orange-colored) wiring and connectors. Do not touch the service plug, which is located under the rear seat. If an accident occurs, do not touch any high-voltage wiring, connectors or parts such as the inverter unit or hybrid vehicle battery assembly. Do not touch the potassium hydroxide electr

In [116]:
import openai
import os
from tenacity import retry, wait_random_exponential, stop_after_attempt, retry_if_not_exception_type

openai.api_key = os.getenv("OPENAI_API_KEY")

# let's make sure to not retry on an invalid request, because that is what we want to demonstrate
@retry(wait=wait_random_exponential(min=1, max=5), stop=stop_after_attempt(3), retry=retry_if_not_exception_type(openai.InvalidRequestError))
def get_embedding(text_or_tokens, model=EMBEDDING_MODEL):
    return openai.Embedding.create(input=text_or_tokens, model=model)["data"][0]["embedding"]

embeddings = []
for string in pdf_strings:
  embeddings.append(get_embedding(string))

df = pd.DataFrame({'text': pdf_strings, 'embedding': embeddings})


In [117]:
import pickle

def export_to_pkl(df: pd.DataFrame):
  # Export to file!
  with open('manuals_embeddings.pkl', 'wb') as f:
    data = pickle.dumps(df)
    f.write(data)

def export_to_csv(df: pd.DataFrame):
  df.to_csv('manuals_embeddings.csv', index=False)


In [118]:
export_to_csv(df)
export_to_pkl(df)